In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [2]:
!pip install -U -q "google-generativeai>=0.8.2"

In [3]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IllvdSBhcmUgYSB0cmFuc2NyaXB0aW9uIHNwZWNpYWxpc3QgZm9yIGZvb2QgY2F0ZXJpbmcgc2VydmljZXMsIGV4cGVydCBhdCByZWZpbmluZyBtZXNzeSB0cmFuc2NyaXB0cyBmcm9tIHNvZnQtcGhvbmUgcmVjb3JkaW5ncy4gIFlvdSB3aWxsIHJlY2VpdmUgYSBtZXNzeSB0cmFuc2NyaXB0IGFzIGlucHV0LiBZb3VyIHRhc2sgaXMgdG8gaW1wcm92ZSBpdCBieSBmb2xsb3dpbmcgdGhlc2Ugc3RlcHM6XFxuXFxuMS4gKipTcGVsbC1jaGVjazoqKiBDYXJlZnVsbHkgcmV2aWV3IHRoZSBlbnRpcmUgdHJhbnNjcmlwdCBmb3Igc3BlbGxpbmcgZXJyb3JzLiBDb3JyZWN0IGFueSBtaXNzcGVsbGluZ3Mgd2hpbGUgcHJlc2VydmluZyB0aGUgb3JpZ2luYWwgbWVhbmluZy4gUGF5IGNsb3NlIGF0dGVudGlvbiB0byBpbmR1c3RyeS1zcGVjaWZpYyB0ZXJtcyByZWxhdGVkIHRvIGZvb2QgY2F0ZXJpbmcuXFxuXFxuMi4gKipSZXBocmFzZToqKiBSZXBocmFzZSBhbnkgYXdrd2FyZCBvciB1bmNsZWFyIHNlbnRlbmNlcyBpbiB0aGUgdHJhbnNjcmlwdC4gRW5zdXJlIHRoZSByZXBocmFzZWQgc2VudGVuY2VzIGFyZSBncmFtbWF0aWNhbGx5IGNvcnJlY3QsIGZsb3cgc21vb3RobHksIGFuZCBtYWludGFpbiB0aGUgb3JpZ2luYWwgY29udGV4dCB3aXRoaW4gdGhlIGZvb2QgY2F0ZXJpbmcgZG9tYWluLlxcblxcbjMuICoqU3VtbWFyaXplOioqIFN1bW1hcml6ZSBhbmQgY29uY2lzZWx5IGNhcHR1cmUgdGhlIG1haW4gcG9pbnRzIG9mIHRoZSByZXBocmFzZWQgdHJhbnNjcmlwdCByZWdhcmRpbmcgZm9vZCBjYXRlcmluZyBzZXJ2aWNlcy4gRm9jdXMgb24gdGhlIGtleSBkZXRhaWxzIGxpa2Ugb3JkZXIgZGV0YWlscywgZGVsaXZlcnkgaW5zdHJ1Y3Rpb25zLCByZWxldmFudCBydWxlcyBvciByZWd1bGF0aW9ucywgYW5kIGFueSBRJkEgd2l0aCB0aGUgY3VzdG9tZXIuIFlvdSAqKm11c3QqKiBpbmNsdWRlIHRoZSBzcGVjaWZpYyBhbnN3ZXJzIHByb3ZpZGVkIGJ5IHRoZSBzYWxlc3BlcnNvbiBzbyB0aGF0IHRoZSBwb2xpY2llcyBhbmQgcHJvY2VkdXJlcyBkaXNjdXNzZWQgY2FuIGNsZWFybHkgYmUgdW5kZXJzdG9vZCB0aHJvdWdoIHRoZSBzdW1tYXJ5IGl0c2VsZi4gTWFrZSBzdXJlIHRvIHJlbW92ZSBjdXN0b21lcidzIG5hbWUsIGNvbnRhY3QgbnVtYmVyLCBvciBlbWFpbCBhZGRyZXNzIGZvciBwcml2YWN5IGNvbmNlcm5zLlxcblxcbjQuICoqQ2xhc3NpZnk6KiogQmFzZWQgb24gdGhlIHN1bW1hcnksIGNsYXNzaWZ5IHRoZSB0ZXh0IHRvIGRldGVybWluZSB0aGUgY2F0ZWdvcnkgb2YgdGhlIHRyYW5zY3JpcHQsIHRoaXMgaW5jbHVkZXM6IFxcXFxcXFwib3JkZXIgcGxhY2VtZW50XFxcXFxcXCIsIFxcXFxcXFwib3JkZXIgYW1lbmRtZW50L21vZGlmaWNhdGlvblxcXFxcXFwiLCBcXFxcXFxcImRlbGl2ZXJ5L2xvZ2lzdGljc1xcXFxcXFwiLCBcXFxcXFxcInByaWNpbmcvcGF5bWVudHNcXFxcXFxcIiwgXFxcXFxcXCJyZWZ1bmRzL2NhbmNlbGxhdGlvbnNcXFxcXFxcIiwgXFxcXFxcXCJjb21wbGFpbnRzL2ZlZWRiYWNrXFxcXFxcXCIsIFxcXFxcXFwiZ2VuZXJhbCBmYXFcXFxcXFxcIiB3aGljaCBjb25zaXN0cyBvZiBnZW5lcmFsIGN1c3RvbWVyIGlucXVpcmllcyAoZS5nLiBwcmljZXMsIGRpc2hlcywgbWVudSBvcHRpb25zLCBldGMuKSBhbmQgXFxcXFxcXCJpcnJlbGV2YW50XFxcXFxcXCIuXFxuXFxuNS4gKipEZXRlcm1pbmU6KiogQmFzZWQgb24gdGhlIG1lc3N5IHRyYW5zY3JpcHQsIGFuZCB0aGUgZm9sbG93aW5nIGxpc3Qgb2Ygb2ZmaWNpYWwgYnJhbmNoZXM6W25lbyBnYXJkZW4sIGNoaWxsaSBtYW5pcywgZGVsaWh1YiwgY2F0ZXIgY28sIGtpbSBwYXJhZGlzZSwgb3JhbmdlIGNsb3ZlLCB0aW5na2F0LCBiZXN0IGNhdGVyaW5nLCBzYXZvdXJ5IGNhdGVyaW5nLCB1bWlzdXNoaSwga3J5c3Rvbl0sIGRldGVybWluZSB0aGUgYnJhbmNoIG9mIHRoZSBjdXJyZW50IHRyYW5zY3JpcHQuIElmIG5vbmUgbWF0Y2gsIHNheSAndW5rbm93bicuXFxcXFxcXCJcXG5cXG5PdXRwdXQgdmFsaWQgSlNPTiB3aXRoIG5vIGV4dHJhIGtleXMuIFlvdSAqKm11c3Qgbm90KiogaW5jbHVkZSBhbnkgaW52YWxpZCBiYWNrc2xhc2hlcy4gRG8gbm90IGVzY2FwZSB0aGUgJCBzeW1ib2wuIElmIHlvdSBuZWVkIHRvIGluY2x1ZGUgYSBkb2xsYXIgc2lnbiwganVzdCB3cml0ZSBpdCBhcyAkLlxcbkRvIG5vdCBpbmNsdWRlIGV4dHJhIGJhY2tzbGFzaGVzIGZvciBVbmljb2RlIGNoYXJhY3RlcnMuIElmIHlvdSBuZWVkIGEgc3BlY2lhbCBjaGFyYWN0ZXIgbGlrZSDCsSwgd3JpdGUgaXQgZGlyZWN0bHkgYXMgwrEgb3IgdXNlIGEgc2luZ2xlIHZhbGlkIEpTT04gZXNjYXBlIHNlcXVlbmNlIFxcXCJcXFxcdTAwQjFcXFwiLlxcblxcbk91dHB1dCB5b3VyIHJlZmluZWQgdHJhbnNjcmlwdCBmb2xsb3dpbmcgdGhpcyBmb3JtYXQ6XFxuKipTdW1tYXJ5OioqIFtUaGUgY29uY2lzZSBzdW1tYXJ5IG9mIHRoZSB0cmFuc2NyaXB0XVxcbioqQ2F0ZWdvcnk6KiogW1RoZSBjbGFzc2lmaWVkIGNhdGVnb3J5IG9mIHRoZSB0cmFuc2NyaXB0XVxcbioqQnJhbmNoOioqIFtUaGUgYnJhbmNoIG5hbWUgaWRlbnRpZmllZF1cXG5cXG5NZXNzeSBUcmFuc2NyaXB0OiBjYW4geW91IHNwZWFrIHRvIG1ycyB5b25nIG1ycyB5b25nIGtlbGx5IGhlcmUgZnJvbSBuZW8gZ2FyZGVuIHlvdSBoYXZlIGxlYXJudCB0byBpbmZvcm0geW91ciBvdGhlciBvbiAxNCBvZiBtZSBvbiAxNCBvZiBhcHJpbCBzZW5kIHNvIHRoaXMgc2VuZCBzbyB3ZSB3aWxsIHN0YXJ0IHRoaXMgYXJvdW5kIDUxNSBjYW4gaSBjaGVjayBzb29uIHdoZW4gdGhleSBjb21lIGF0IDUxNSBob3cgbG9uZyBkbyB0aGV5IG5lZWQgdG8gc3RhcnQgdXAgdGhleSBzdGFydCB1cCBhdCBoYWxmIGFuIGhvdXIgNTE1IGFuZCB0aGUgZm9vZCByZWFkeSBieSA1NDUgdGhleWxsIGJlIHBsdXMgb3IgbWludXMgMzAgbWludXRlcyBpbiB0aGF0IGNhc2Ugd2hlbiBhIGNsaWVudCBjb21lcyBiYWNrIHdlIGNoYW5nZSB0byA1IHBtIGFuZCB0aGUgZm9vZCByZWFkeSBieSA1MzAgcGx1cyBvciBtaW51cyAzMCBtaW51dGVzIHRoZW4gdGhlIGNvbGxlY3Rpb24gdGltZSBtIGtub3cgd2hhdCB0aW1lIHdpbGwgcHJlZmVyIG1heGltdW0gZm9yIG91ciB1bnRpbCA5IHBtIDkgcG0gb2ggdGhlbiA5IHBtIDkgcG0gaXMgZm9yIHlvdSBodWggZ29vZCBzbyBhbHRob3VnaCB3ZWxsIGJlIHNlbnQgdG8gZG8geW91IHByb3ZpZGUgYSBjaGFuY2UgYW5kIHRhYmxlIG9yIHQgYWRkaXRpb25hbCBjaGFuY2UgdGFibGUgaXMgdW5kZXIgcmVudGFsIGhvdyBtdWNoIHdvdWxkIHRoYXQgYmUgdGhlc2UgYXJlIHJpZ2h0IHRoZW4gdGhlIHRhYmxlIGlzIGdvb2QgdGFnIG9uZSB0YWJsZSBjYW4gc2l0IGFib3V0IDYgcGVvcGxlIGNoYXJnZSBoZXIgMTUgYW5kIHRoZSB0d28gYXJlIGJhZCByZXN0IDEgcGVyIHBpZWNlIGkgdGFraW5nIHJlYWwgd2hldGhlciBpIHVzZSBhIHRhYmxlIG9yIHQgdGhleSBoYXZlIHVnaCB0YWJsZSBhbmQgY2hlY2sgeW91ciBmdW5jdGlvbiByb29tIGkgdGFuayBoYXZlIHJpZ2h0IGEgcGhvbmUgYSBwaG9uZSBpZiB5b3Ugc2NhbiBpdCB1Z2ggaXQgY2FuIHJlbnQgYmUgMSBvciAyIDEgb3IgMiBpIHJlbnQgb25lIHRhYmxlIDEgZXh0cmEgdGFibGUgaHVoIGhvdyBhYm91dCB0aGUgdHdvIGkgYWN0dWFsbHkgaGF2ZSBsaWtlIDYgb3IgNiAyIG15c2VsZiBob3cgYWJvdXQgdGhlIGZ1bmN0aW9uIHJvb20gZ290IGNoZWNrIGkgaGF2ZSB0aGVuIG5lZWQgZm9yIHRoZSBjaGVjayBpIHdhbnQgdG8gcmVudCAxMCBtaW5pbXVtIDEwIHBpZWNlcyBoIG9yIDEgcGllY2UgMSBwZXIgcGllY2UgaSB3YW50IDEgcGVyIHBpZWNlIG1pbmltdW0gMTAgcGllY2VzIGkgZ2V0IDEwIDEwIDEgZXh0cmEgdGFibGUgYW5kIDEwIDIgYmVmb3JlIHRoZSBlbmVteSBjb21mb3J0cyBhZGRpdGlvbmFsIDI1IHJpZ2h0IGNvcnJlY3QgY29ycmVjdCBzbyBsZXQgbWUgY29uZmlybSB5b3VyIGFkZHJlc3MgZmlyc3QgYWRkcmVzcyB3aWxsIGJlIHNlbnQgdG8gYmxvc3NvbSByZXNpZGVuY2VzIGZ1bmN0aW9uYWwgYXQgbGV2ZWwgMSByaWdodCAxMyBzaW5nIGkgd3JvdGUgaSBrbm93IGZ1bmN0aW9uYWwgYXQgbGV2ZWwgMSBoIHNvIHRoZSBmdW5jdGlvbiBuZWVkIHRvIHAgYW55IHN0IGtpc3MgdGhlcmUgYSBkaXJlY3QgYWNjZXNzIHRvIGxldmVsIDEgZnVuY3Rpb25hbCByaWdodCBoIGNvcnJlY3QgaSBtZWFuIHlvdSBndXlzIHdvdWxkIGhhdmUgY29tZSB0byBoZXJlIGJlZm9yZSByaWdodCBvaCBoIHdlIGNvbWUgdGhlcmUgYmVmb3JlIGJ1dCBqdXN0IHRvIGRvdWJsZSBjaGVjayBpIGRvbnQgbmVlZCB0byBwIGFueSBzdCBraXNzIG5lZWQgZ29vZCAxMCBwcm9ibGVtIHNvIG90aGVyIHdvdWxkIGJlIGZvciA0MCBwZW9wbGUgdW5kZXIgdGhlIG5ldyBibG9zc29tIG1hbnVhbCB0aGVyZSBhZGRpdGlvbmFsIGZvciAxMDAgcGllY2VzIHJpZ2h0IGggYWxsIHRoZSBmb29kIHdpbGwgYmUgd2FybWVyIHJpZ2h0IHcgY29ycmVjdCB3ZSBqdXN0IGNvIG9uZSBob3VyIGJlZm9yZSBhbmQgdGhlIGZvb2QgY29tZXMgd2l0aCB3YXJtZXIgc3RpbGwgaG90IGJlc3QgY29uc3VtZSB0cmVhdCBvdXIgcmVhbCBpc3N1ZSBhbmQgdGhlbiB0aGVyZSB0aGUgY3V0bGVyeSByaWdodCB0aGUgdGFibGUgZm9yIGZvb2Qgd2FybWVyIHV0ZW5zaWwgbGlrZSBmb3VyIHNwb29uIHVudGlsIGl0cyBhbGwgMjAwNyBodWggdmVlciB0aGlzIG9uZSBpcyBmb3IgYm90aCBkIG9yIHdvcmtvdXQgZXZlbnQgdCByZWFsbHkgYmlydGggeWFtYWhhIGp1c3QgYSBnYXRoZXJpbmcganVzdCBsb25nZXIgbG9uZ2VyIGFuZCBzbWFsbGVyIGp1c3QgYSBzaW5nbGUgY2VsZWJyYXRpb24ganVzdCBnYXRoZXJpbmcgYWxsIHRoZSB3IGggZ2F0aGVyaW5nIGhhIGNhbnQgZ2V0IHByb2JsZW0gc28gbXIgY29uZyBpIHdvdWxkIHMgdGhlIGNvbmZpcm1hdGlvbiBlbWFpbCB0b2dldGhlciB3aXRoIGEgcGVudCBsaW5rIGphZGUgYW5kIGJvbGUgZGVuYXJpIHNlYSBpdGVtIGEgYm9sZSBwdWxhIG5vIHRlcnJhaW4gYm9sZSBib2xlIGFkYSBubyBsYWluIGEgYWthbiBrZXlwYWQgYW5kIGEgYWthbiBrZXlwYWQgYW5kIGFkYSBubyBsYWluIHVudHVjayBrb2FsYSBrb2FsYSBubyBsYWluIHVudHVjayBrb2FsYSBrb2FsYSBubyBsYWluIHVudHVjayBrb2FsYSBvbmx5IGZvciBiYWJ5IHdvbWFuIGRhbiBubyBwZXJ0YWluIGphZGUga29hbGEgbm8gYW5kIGtvYWxhIGEgdGFrZW4gYm9sZSBhIHRha2VuIGtleXBhZCBhbmQgdGVybXMgY2FzaCB0ZXJtcyBjYXNoIHRlcm1zIGNhc2gifV19LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOlt7InRleHQiOiJgYGBqc29uXFxue1xcbiAgXFxcIlN1bW1hcnlcXFwiOiBcXFwiQSBjYXRlcmluZyBvcmRlciB3YXMgcGxhY2VkIHdpdGggTmVvIEdhcmRlbiBmb3IgYW4gZXZlbnQgYXQgQmxvc3NvbSBSZXNpZGVuY2VzLiBUaGUgb3JkZXIgaXMgZm9yIDQwIHBlb3BsZSwgd2l0aCBhZGRpdGlvbmFsIGN1dGxlcnkgZm9yIDEwMCBwZW9wbGUuICBUaGUgZXZlbnQgaXMgYSBnYXRoZXJpbmcsIG5vdCBhIHdlZGRpbmcgb3IgY29ycG9yYXRlIGV2ZW50LiAgRm9vZCBkZWxpdmVyeSBpcyBzY2hlZHVsZWQgZm9yIDU6MDAgUE0sIHdpdGggdGhlIGZvb2QgcmVhZHkgYnkgNTozMCBQTSAowrEzMCBtaW51dGVzKS4gT25lIGV4dHJhIHRhYmxlIChmb3IgNiBwZW9wbGUpIGFuZCAxMCBhZGRpdGlvbmFsIGNoYWlycyB3aWxsIGJlIHJlbnRlZC4gVGhlIHRvdGFsIGNvc3QgZm9yIHRhYmxlIGFuZCBjaGFpciByZW50YWwgaXMgJDE1IGZvciB0aGUgdGFibGUgYW5kICQxMCBmb3IgdGhlIGNoYWlycyAoJDEwIHggMTAgY2hhaXJzID0gJDEwMCkuIFRoZSBkZWxpdmVyeSBhZGRyZXNzIGlzIEJsb3Nzb20gUmVzaWRlbmNlcywgTGV2ZWwgMS4gVGhlcmUncyBkaXJlY3QgYWNjZXNzIHRvIExldmVsIDEuICBUaGUgZm9vZCB3aWxsIGJlIHNlcnZlZCB3aXRoIHdhcm1lcnMgdG8gZW5zdXJlIGl0IHN0YXlzIGhvdC4gVGhlIG9yZGVyIGluY2x1ZGVzIGN1dGxlcnksIHBsYXRlcywgc3Bvb25zLCBldGMuIFBheW1lbnQgd2lsbCBiZSBtYWRlIGluIGNhc2guXFxcIixcXG4gIFxcXCJDYXRlZ29yeVxcXCI6IFxcXCJvcmRlciBwbGFjZW1lbnRcXFwiLFxcbiAgXFxcIkJyYW5jaFxcXCI6IFxcXCJuZW8gZ2FyZGVuXFxcIlxcbn1cXG5gYGBcXG4ifV19XQ==' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjIwNDh9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "You are a transcription specialist for food catering services, expert at refining messy transcripts from soft-phone recordings.  You will receive a messy transcript as input. Your task is to improve it by following these steps:\\n\\n1. **Spell-check:** Carefully review the entire transcript for spelling errors. Correct any misspellings while preserving the original meaning. Pay close attention to industry-specific terms related to food catering.\\n\\n2. **Rephrase:** Rephrase any awkward or unclear sentences in the transcript. Ensure the rephrased sentences are grammatically correct, flow smoothly, and maintain the original context within the food catering domain.\\n\\n3. **Summarize:** Summarize and concisely capture the main points of the rephrased transcript regarding food catering services. Focus on the key details like order details, delivery instructions, relevant rules or regulations, and a

## Call `generate_content`

In [4]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [5]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

You are a transcription specialist for food catering services, expert at refining messy transcripts from soft-phone recordings.  You will receive a messy transcript as input. Your task is to improve it by following these steps:\n\n1. **Spell-check:** Carefully review the entire transcript for spelling errors. Correct any misspellings while preserving the original meaning. Pay close attention to industry-specific terms related to food catering.\n\n2. **Rephrase:** Rephrase any awkward or unclear sentences in the transcript. Ensure the rephrased sentences are grammatically correct, flow smoothly, and maintain the original context within the food catering domain.\n\n3. **Summarize:** Summarize and concisely capture the main points of the rephrased transcript regarding food catering services. Focus on the key details like order details, delivery instructions, relevant rules or regulations, and any Q&A with the customer. You **must** include the specific answers provided by the

In [8]:
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# Import the data

In [10]:
data = []

my_file = open("/content/drive/MyDrive/NGC/all_spell_checked.txt", "r")

# reading the file
line = my_file.read()

# splitting the text it further when '\n' is seen.
line = line.split("\n")

data.extend(line)
my_file.close()

print(len(data)) # printing the data

6871


# Start generating response
- Model: Gemini-1.5-flash

In [17]:
"""
Install an additional SDK for JSON schema support Google AI Python SDK

$ pip install google.ai.generativelanguage
"""

import os
import google.generativeai as genai
from google.ai.generativelanguage_v1beta.types import content

# genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 2048,
  "response_schema": content.Schema(
    type = content.Type.OBJECT,
    enum = [],
    required = ["summary", "category", "branch"],
    properties = {
      "summary": content.Schema(
        type = content.Type.STRING,
      ),
      "category": content.Schema(
        type = content.Type.STRING,
      ),
      "branch": content.Schema(
        type = content.Type.STRING,
      ),
    },
  ),
  "response_mime_type": "application/json",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  system_instruction="You are an expert in English and correcting transcripts for a series of soft-phone recording from a catering service.",
)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        "You are a transcription specialist for food catering services, expert at refining messy transcripts from soft-phone recordings.  You will receive a messy transcript as input. Your task is to improve it by following these steps:\n\n1. **Spell-check:** Carefully review the entire transcript for spelling errors. Correct any misspellings while preserving the original meaning. Pay close attention to industry-specific terms related to food catering.\n\n2. **Rephrase:** Rephrase any awkward or unclear sentences in the transcript. Ensure the rephrased sentences are grammatically correct, flow smoothly, and maintain the original context within the food catering domain.\n\n3. **Summarize:** Summarize and concisely capture the main points of the rephrased transcript regarding food catering services. Focus on the key details like order details, delivery instructions, relevant rules or regulations, and any Q&A with the customer. You **must** include the specific answers provided by the salesperson so that the policies and procedures discussed can clearly be understood through the summary itself. Make sure to remove customer's name, contact number, or email address for privacy concerns.\n\n4. **Classify:** Based on the summary, classify the text to determine the category of the transcript, this includes: \\\"order placement\\\", \\\"order amendment/modification\\\", \\\"delivery/logistics\\\", \\\"pricing/payments\\\", \\\"refunds/cancellations\\\", \\\"complaints/feedback\\\", \\\"general faq\\\" which consists of general customer inquiries (e.g. prices, dishes, menu options, etc.) and \\\"irrelevant\\\".\n\n5. **Determine:** Based on the messy transcript, and the following list of official branches:[neo garden, chilli manis, delihub, cater co, kim paradise, orange clove, tingkat, best catering, savoury catering, umisushi, kryston], determine the branch of the current transcript. If none match, say 'unknown'.\\\"\n\nOutput valid JSON with no extra keys. You **must not** include any invalid backslashes. Do not escape the $ symbol. If you need to include a dollar sign, just write it as $.\nDo not include extra backslashes for Unicode characters. If you need a special character like ±, write it directly as ± or use a single valid JSON escape sequence \"\\u00B1\".\n\nOutput your refined transcript following this format:\n**Summary:** [The concise summary of the transcript]\n**Category:** [The classified category of the transcript]\n**Branch:** [The branch name identified]\n\nMessy Transcript: can you speak to mrs yong mrs yong kelly here from neo garden you have learnt to inform your other on 14 of me on 14 of april send so this send so we will start this around 515 can i check soon when they come at 515 how long do they need to start up they start up at half an hour 515 and the food ready by 545 theyll be plus or minus 30 minutes in that case when a client comes back we change to 5 pm and the food ready by 530 plus or minus 30 minutes then the collection time m know what time will prefer maximum for our until 9 pm 9 pm oh then 9 pm 9 pm is for you huh good so although well be sent to do you provide a chance and table or t additional chance table is under rental how much would that be these are right then the table is good tag one table can sit about 6 people charge her 15 and the two are bad rest 1 per piece i taking real whether i use a table or t they have ugh table and check your function room i tank have right a phone a phone if you scan it ugh it can rent be 1 or 2 1 or 2 i rent one table 1 extra table huh how about the two i actually have like 6 or 6 2 myself how about the function room got check i have then need for the check i want to rent 10 minimum 10 pieces h or 1 piece 1 per piece i want 1 per piece minimum 10 pieces i get 10 10 1 extra table and 10 2 before the enemy comforts additional 25 right correct correct so let me confirm your address first address will be sent to blossom residences functional at level 1 right 13 sing i wrote i know functional at level 1 h so the function need to p any st kiss there a direct access to level 1 functional right h correct i mean you guys would have come to here before right oh h we come there before but just to double check i dont need to p any st kiss need good 10 problem so other would be for 40 people under the new blossom manual there additional for 100 pieces right h all the food will be warmer right w correct we just co one hour before and the food comes with warmer still hot best consume treat our real issue and then there the cutlery right the table for food warmer utensil like four spoon until its all 2007 huh veer this one is for both d or workout event t really birth yamaha just a gathering just longer longer and smaller just a single celebration just gathering all the w h gathering ha cant get problem so mr cong i would s the confirmation email together with a pent link jade and bole denari sea item a bole pula no terrain bole bole ada no lain a akan keypad and a akan keypad and ada no lain untuck koala koala no lain untuck koala koala no lain untuck koala only for baby woman dan no pertain jade koala no and koala a taken bole a taken keypad and terms cash terms cash terms cash",
      ],
    },
    {
      "role": "model",
      "parts": [
        "```json\n{\n  \"Summary\": \"A catering order was placed with Neo Garden for an event at Blossom Residences. The order is for 40 people, with additional cutlery for 100 people.  The event is a gathering, not a wedding or corporate event.  Food delivery is scheduled for 5:00 PM, with the food ready by 5:30 PM (±30 minutes). One extra table (for 6 people) and 10 additional chairs will be rented. The total cost for table and chair rental is $15 for the table and $10 for the chairs ($10 x 10 chairs = $100). The delivery address is Blossom Residences, Level 1. There's direct access to Level 1.  The food will be served with warmers to ensure it stays hot. The order includes cutlery, plates, spoons, etc. Payment will be made in cash.\",\n  \"Category\": \"order placement\",\n  \"Branch\": \"neo garden\"\n}\n```\n",
      ],
    },
  ]
)

In [25]:
chat_session.history

[parts {
   text: "You are a transcription specialist for food catering services, expert at refining messy transcripts from soft-phone recordings.  You will receive a messy transcript as input. Your task is to improve it by following these steps:\n\n1. **Spell-check:** Carefully review the entire transcript for spelling errors. Correct any misspellings while preserving the original meaning. Pay close attention to industry-specific terms related to food catering.\n\n2. **Rephrase:** Rephrase any awkward or unclear sentences in the transcript. Ensure the rephrased sentences are grammatically correct, flow smoothly, and maintain the original context within the food catering domain.\n\n3. **Summarize:** Summarize and concisely capture the main points of the rephrased transcript regarding food catering services. Focus on the key details like order details, delivery instructions, relevant rules or regulations, and any Q&A with the customer. You **must** include the specific answers provided 

In [21]:
import json
import time

final_json = []

for i in range(len(data)):
    response = chat_session.send_message(data[i])
    response_json = json.loads(response.text)

    # Append more fields
    response_json['content_type'] = 'text'
    response_json['raw_transcript'] = data[i]

    final_json.append(response_json)

In [22]:
final_json

[{'branch': 'kim paradise',
  'category': 'general faq',
  'summary': "The customer inquired about the pricing of Kim Paradise's King Card model 5, specifically whether the price differs between plastic containers and disposable ones. The salesperson clarified that the price is currently the same for both options, unlike in a previous instance where there was a price difference. The customer mentioned a previous order of 3000 units.",
  'content_type': 'text',
  'raw_transcript': 'kim paradise morning id like to ask for a king card is there a difference in price whether it is plastic containers or disposable king card model 5 price tank of a disposable same price additional touch last time there was a difference right h for my growth container last time we got 3000 10 but w its a same price as well'},
 {'branch': 'kim paradise',
  'category': 'order amendment/modification',
  'summary': 'The customer called to discuss their contract renewal. Their previous contract was from 14/15, with

In [23]:
with open("/content/drive/MyDrive/NGC/summaries.json", "w", encoding="utf-8") as f:
    json.dump(final_json, f, ensure_ascii=False, indent=4)